In [1]:
import langchain.llms as llms
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains.summarize import load_summarize_chain
from langchain.llms import LlamaCpp
import pandas as pd
import os
import textwrap

In [2]:
llm = LlamaCpp(
    model_path="llama.cpp/models/rinna-youri-7b-chat-q4_K_M.gguf",
    # model_path=r"llama.cpp/cpp\models\llama-2-7b-chat.Q4_K_M.gguf",
    input={
        "max_tokens": 32,
        "stop": ["System:", "User:", "Assistant:", "\n"],
    },
    verbose=True,
    temperature=1,
    n_ctx=2048
)
chain = load_summarize_chain(llm, chain_type="map_reduce")


c:\Users\onowa\miniconda3\envs\vir_env\lib\site-packages\langchain\utils\utils.py:159: UserWarning: WARNING! input is not default parameter.
                input was transferred to model_kwargs.
                Please confirm that input is what you intended.
  warnings.warn(
AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [3]:
def summarize_pdf(chain, path):
    loader = PyPDFLoader(path)
    pages = loader.load_and_split()
    return chain.run(pages)

In [4]:
from langchain.document_loaders import TextLoader
def summarize_text(chain, path):
    # loader = PyPDFLoader(path)
    # pages = loader.load_and_split()
    # return chain.run(pages)
    loader = TextLoader(path)
    data = loader.load()
    # print(data)
    return chain.run(data)


In [5]:
# summarize_pdf(chain, r"C:\Users\onowa\OneDrive\Documents\The story follows elven mage Frieren.pdf") 
# r"C:\Users\onowa\OneDrive\Documents\P-010_hikae_TZ-00441267_20230709.pdf")
summarize_text(chain, r"C:\Users\onowa\OneDrive\Documents\フリーレンあらすじ.txt")

Llama.generate: prefix-match hit


'\n1. ヒンメルは魔王との決戦で仲間を失い、50年前パーティを解散させた。\n2. フリーレンが他界してからもう少し、今回再会することになった4人の勇者は魔法や知識を学ぶ時間が短くなりつつあった。\n3. 10年後、フェルンは大きな仕事で自分の身元を明かす準備をしているハイターと出会う。'

https://github.com/DrEntropy/PDFSummary/blob/2fe7cd1d1d15f6e3f208d0705aff11ea995e56a8/PDF_summary.ipynb#L20

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator = "\n\n",  # セパレータ
    chunk_size = 11,  # チャンクの文字数
    chunk_overlap = 0,  # チャンクオーバーラップの文字数
)
textlist = ['PFRの家庭向けロボ、大規模言語モデル経由で操作 生成コードはLuaを採用、将来はマルチモーダル型へ', 'LINEヤフーが人体CG向け生成AI技術 役割を指定して2人の相互動作を出力', '〔第１０１回〕\u3000ＣＥＡＴＥＣ2023：大規模言語モデルＰＬａＭｏ、\u3000ＭＮ−Ｃｏｒｅ、3Ｄ空間スキャン', 'ロボの大規模言語モデル応用が次のフェーズへ 動作計画の拘束条件をLLMで生成、学習データ不要', '類似文書検索と大規模言語モデルを用いた事故記録の解析']
def summarize_list(chain, path):
    chain = load_summarize_chain(llm, chain_type="map_reduce")
    # chain.run(docs)
    return chain.run(path)
# summarize_text(chain, textlist)
texts = text_splitter.create_documents(textlist)
summaries = [summarize_list(chain, file) for file in texts]
summaries[0]

In [13]:
from langchain.docstore.document import Document
# 分割した長文を書く文章ごとにDocumentオブジェクト化
docs = [Document(page_content=t) for t in textlist]

chain = load_summarize_chain(
    llm,
    chain_type="map_reduce",
    verbose=True,
)

output = chain(
    inputs=docs,
    return_only_outputs=True,
)["output_text"]

print(output)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"PFRの家庭向けロボ、大規模言語モデル経由で操作 生成コードはLuaを採用、将来はマルチモーダル型へ"


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"LINEヤフーが人体CG向け生成AI技術 役割を指定して2人の相互動作を出力"


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"〔第１０１回〕　ＣＥＡＴＥＣ2023：大規模言語モデルＰＬａＭｏ、　ＭＮ−Ｃｏｒｅ、3Ｄ空間スキャン"


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"ロボの大規模言語モデル応用が次のフェーズへ 動作計画の拘束条件をLLMで生成、学習データ不要"


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"類似文書検索と大規模言語モデルを用いた事故記録の解析"


CONCISE SUMMARY:


Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"

Home Assistant, the digital assistant that PFR offers to consumers, has been redesigned and expanded to include a large language model.  The new design is based on LUA and will allow multiple languages in the future.



LINEとYahooは、架空のキャラクターやエンティティが、生成AIモデルによって生成された2人の相互作用を表示する新しい技術を開発しました。  この研究は、架空のキャラクターである「Pony」と「Joe」について説明されており、両者が人間型コンピュータ生成されたキャラクターを使って相互作用を持っています。  この技術は、単一の入力フィールドと複数の入力レイヤーによる生成モジュールから構成されており、1つ以上の入力があり、相互作用を持った2人（またはグループ）に特化しています。  これによ



 A new application for large language models (LLMs) will allow people to create task plans that do not require any data, and which can adapt to changes in the environment.  LLMs have already been put to use planning complex operations like manufacturing and surgery, but this latest application is less restricted as it uses “generative pre-trained transformer” (GPT) language models to synthesize task 

Llama.generate: prefix-match hit



> Finished chain.

> Finished chain.


PFR announces its new digital assistant Home Assistant, which now features a large language model that can answer general questions (similar to Google Search or Bing) as well as create task plans based on natural language input.


In [15]:
print(output)



PFR announces its new digital assistant Home Assistant, which now features a large language model that can answer general questions (similar to Google Search or Bing) as well as create task plans based on natural language input.
